Rebuilding Steps, Example Network, Integrated in SNMan 'Multi' Rebuild
====================
xxx

In [1]:
import warnings
warnings.filterwarnings('ignore')

import copy
import os
import sys
import networkx as nx
import numpy as np
import shutil

import snman
from snman.constants import *
from snman import osmnx_customized as oxc
from snman import *

PERIMETER = 'db'

data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
#export_path = os.path.join(data_directory, 'outputs', PERIMETER)
export_path = os.path.join(data_directory, 'outputs', 'db_sprint01_var02')

CRS_internal = 2056      # for Zurich

Loading data
------------
Loads the prepared simplified street graph, as well as other datasets such as perimeters, rebuilding regions and measurement regions

In [2]:
print('Load street graph')
G = snman.io.load_street_graph(
    os.path.join(process_path, 'street_graph_edges.gpkg'),
    os.path.join(process_path, 'street_graph_nodes.gpkg'),
    crs=CRS_internal
)

street_graph.organize_edge_directions(G)

Load street graph


In [3]:
print('Load rebuilding regions')
# Polygons that define which streets will be reorganized
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    #os.path.join(process_path, 'rebuilding_regions.gpkg'),
    os.path.join(inputs_path, 'rebuilding_regions', 'rebuilding_regions.gpkg'),
    #only_active=True,
    projects=['db_sprint01_var01'],
    #filter_ids=['db_main_roads'],
    #filter_ids=['db_main_roads', 'db_local_01'],
    #filter_ids=['db_local_09'],
    crs=CRS_internal
)

if 1:
    print('Load access needs')
    access_needs = snman.io.load_access_needs(
        os.path.join(inputs_path, 'switzerland', 'zurich', 'statpop', 'statpop17_zurich_city_agg_on_buildings.gpkg'),
        CRS_internal
    )
    access_needs['parking_spots_needed'] = access_needs['residents'] / 60
    
if 1:
    print('Load lane edits')
    lane_edits = snman.io.load_lane_edits(
        os.path.join(inputs_path, 'lane_edits', 'lane_edits.gpkg')
    )
    snman.enrichment.match_lane_edits(
        G, lane_edits, lanes_key=KEY_FORCED_GIVEN_LANES_DESCRIPTION,
        max_dist=200, max_dist_init=500, max_lattice_width=5,
    )
    
if 1:
    print('Load hierarchy edits')
    hierarchy_edits = snman.io.load_hierarchy_edits(
        os.path.join(inputs_path, 'hierarchy_edits', 'hierarchy_edits.gpkg')
    )
    snman.enrichment.match_hierarchy_edits(
        G, hierarchy_edits,
        max_dist=200, max_dist_init=500, max_lattice_width=5,
    )

Load rebuilding regions
Load access needs
Load lane edits
Load hierarchy edits


In [4]:
from snman.rebuilding import *

HLAs = multi_rebuild_regions(
    G, rebuilding_regions_gdf, access_needs,
    save_steps_path=os.path.join(export_path, 'steps'),
    save_steps_scaling_factor=2,
    gravity_iterations=10,
)
HLAs = list(filter(lambda HLA: HLA is not None, HLAs))

truncating street graph
done truncating
GIVEN LANES
REMOVE PARKING
iteration 1
REMOVE CAR LANES
iteration 1
fixed (119, 322, '119-322-0-0')
iteration 2
removed (322, 119, '119-322-0-0')
iteration 3
fixed (857, 859, '857-859-0-0')
iteration 4
removed (859, 857, '857-859-0-0')
iteration 5
removed (378, 1265, '378-1265-0-0')
iteration 6
fixed (1265, 378, '378-1265-0-0')
iteration 7
fixed (1164, 854, '854-1164-0-0')
iteration 8
removed (1518, 1622, '1518-1622-0-0')
iteration 9
fixed (1622, 1518, '1518-1622-0-0')
iteration 10
removed (1601, 1604, '1601-1604-0-0')
iteration 11
fixed (1604, 1601, '1601-1604-0-0')
iteration 12
removed (1604, 849, '849-1604-0-0')
iteration 13
removed (849, 1498, '849-1498-0-0')
iteration 14
removed (1498, 848, '848-1498-0-0')
iteration 15
removed (1564, 1601, '1564-1601-0-0')
iteration 16
fixed (854, 34, '34-854-0-0')
iteration 17
fixed (2, 322, '2-322-0-0')
iteration 18
removed (119, 2, '2-119-0-0')
iteration 19
removed (16, 859, '16-859-0-0')
iteration 20
rem

In [5]:
import networkx as nx

Hs,Ls,As = zip(*HLAs)
A = nx.compose_all(As)
L = nx.compose_all(Ls)

snman.io.export_HLA(os.path.join(export_path, 'steps'), '_debug', L=L, A=A)

In [6]:
print('Reorder lanes')
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION)
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)

print('Normalize cycling infrastructure')
snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION)
snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION_AFTER)

snman.street_graph.organize_edge_directions(G)

Reorder lanes
Normalize cycling infrastructure


In [7]:
print('Add elevation')
G = oxc.elevation.add_node_elevations_raster(
    G,
    os.path.join(inputs_path, 'copernicus_dem', 'Copernicus_DSM_10_N47_00_E009_00', 'DEM', 'Copernicus_DSM_10_N47_00_E009_00_DEM.tif'),
    raster_crs=2056,
    graph_crs=CRS_internal,
    cpus=1
)
G = oxc.elevation.add_edge_grades(G, add_absolute=False)

Add elevation


In [8]:
if 0:

    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION)
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width', lanetypes={LANETYPE_CYCLING_LANE})
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width')
    snman.street_graph.lane_graph_to_street_graph(
        G, L,
        KEY_LANES_DESCRIPTION
    )
    
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width', lanetypes={LANETYPE_CYCLING_LANE})
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width')
    snman.street_graph.lane_graph_to_street_graph(
        G, L,
        KEY_LANES_DESCRIPTION_AFTER
    )

In [9]:
snman.space_allocation.generate_lane_stats(G, lanes_attribute=KEY_LANES_DESCRIPTION)
snman.space_allocation.generate_lane_stats(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)

In [11]:
if 1:
    print('export G')
    snman.io.export_street_graph(
        G,
        os.path.join(export_path, 'G_edges.gpkg'),
        os.path.join(export_path, 'G_nodes.gpkg')
    )

if 1:
    print('export L before')
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION)
    snman.io.export_lane_geometries(
        L,
        os.path.join(export_path, 'lane_geometries_edges.gpkg'),
        os.path.join(export_path, 'lane_geometries_nodes.gpkg'),
        scaling=2
    )

if 1:
    print('export L after')
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)
    snman.io.export_lane_geometries(
        L,
        os.path.join(export_path, 'lane_geometries_rebuilt_edges.gpkg'),
        os.path.join(export_path, 'lane_geometries_rebuilt_nodes.gpkg'),
        scaling=2
    )

export G
export L before
export L after
(362, 1607, '362-1607-0-1') geometry offset failed
(1607, 366, '366-1607-0-1') geometry offset failed
(404, 994, '404-994-0-1') geometry offset failed
